<a href="https://colab.research.google.com/github/priyangshu-datta/master-project/blob/main/llm_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Editable
1. Prompt
1. Queries
1. Keywords
1. Temperature
1. Rerank ?
1. Rerank query
1. DMDD ID range

In [ ]:
# @title Pip statements
%%capture
%pip install sentence_transformers # InstructorEmbedding
%pip install flashrank
%pip install black[jupyter] --quiet
%pip install nltk
%pip install python-Levenshtein
%pip install pydash
%pip install icecream

In [ ]:
# @title Import statements
from pprint import pprint
import json
import re
import string
from flashrank import Ranker, RerankRequest
from sentence_transformers import SentenceTransformer, util
import tqdm
import pandas as pd
import numpy as np
from google.colab import data_table
import time
import google.ai.generativelanguage as glm
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from google.oauth2 import service_account
import nltk
from nltk.corpus import stopwords
from Levenshtein import distance, ratio
import pydash as py_
import pickle
from icecream import ic

In [ ]:
data_table.enable_dataframe_formatter()

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

with open("./drive/MyDrive/Thesis/data//mini_modified_DMDD.json", "r") as file:
    dmdd = json.load(file)

Mounted at /content/drive


In [ ]:
# mini_dmdd = {}

# for _ in dmdd:
#     mini_dmdd[_] = {}
#     for __ in range(0,1000):
#         mini_dmdd[_][f"{__}"] = dmdd[_][f"{__}"]

In [ ]:
def get_text(data, dmdd_id):
    return data["text"][f"{dmdd_id}"]

def get_true_datasets(dmdd, dmdd_id):
    return list(dmdd["datasets"][f"{dmdd_id}"].keys())

def get_sectioned_text(data, dmdd_id):
    sections = data["sections_spans"][f"{dmdd_id}"]
    for section in sections:
        sections[section] = get_text(data, dmdd_id)[slice(*sections[section])]
    return sections

In [ ]:
re.sub(r"([A-Z][a-z]+)\.(?: ?(\d+) ?\.( [A-Z]))?", r"\1[dot] \2[dot] \3", "Experiments on the Human3.6M dataset show")

'Experiments on the Human3.6M dataset show'

In [ ]:
sub_ci = lambda x, y: py_.partial(re.sub, x, y, flags=re.IGNORECASE)
sub_cs = lambda x, y: py_.partial(re.sub, x, y)
emoticons = r"\(>\.>\)|\(\^\.\^ゞ\)|\(\^_\^\)Y|:\-\)|;\-@|;\-\^|\(>\.<\)\(\^\.\^\)|\(\^_\-\)/\~\~|:\^|;\(|\(\^_\^\)/|\(ToT\)|:\-\^|\(\^\^ゞ|:\-=|:\-\#|;\-\[|\(>_>\)|:\-D|\(>\.<\)|\(\^o\^\)丿|:\-\.|:P|\(\^_\^\)\-☆|\(\^_\^\)w|;\\|:\-o|;\-C|;\-S|\(\^_\^\)v|:\-C|\(>\.<\)b|\(\*_\*\)|\(\-_\-;\)|;P|;=|\(\^_\-\)b|\(\^o\^\)|:\-P|:\#|\(\*\^\.\^\*\)|>:\[|\(\^_\-\)/\~|:\$|\(\^ω\^\)|:\-\{|:'\-\(|\(\^_\-\)\-☆|\(\-_\-\)|x\-\)|:\-X|:X|\(\*O\*\)|\(\*\^_\^\*\)|\(<_<\)|\(ーー;\)|;\-\#|:\*|;\-P|;\-!|:@|\(\^_\-\)Y|:/|\(\^_\-\)W|:\-0|\(\~_\~\)|;/|:!|;\-D|X\-\)|;\-/|;\-=|\(@_@\)|\(°\~°\)|\(\^_\^メ\)|:'\(|8\-\)|\(°u°\)|;\-\(|:\-\(|:\\|:D|;\-\\|\(>_<\)|\(\^ε\^\)|\(\^_\^\)b|:O|\(\^з\^\)|:\-\&|:=|O:\-\)|\(\^\.\^\)|:\-!|;'\-\)|\('\-'\)|\(\._\.\)|:\-<|;O|\(\^人\^\)|\(\^_\^\)|\(°\-°\)|:'\)|;\-\)|\(\^\-\^\)|;\-\$|\(\^\-\^\)b|\(,_,\)|\(\^_\-\)w|;\-\&|;D|:\-\||\(°_°\)|:S|:\-\\|>:D|;\-\{|\(\^\.\^\)y|\(\^_\-\)d|\(°\.°\)|\(\^_\^\)/\~|:\-\[|:\-/|\(\^_\^\*\)|:\&|;\-<|;'\)|:\)|;\)|;\*|\(\^_\-\)|:\-O|;'\-\(|:\-S|;\-O|:\(|B\-\)|\(\~_\^\)|;@|\(\^\-\^ゝ゛\)|\(\^_\^\)W|;\^|;S|\(°o°\)|\(\^O\^\)|\(\*o\*\)|\(>﹏<\)|;\||;\&|\(\^_\^\)/\~\~|:\||>:\)|\(\^_\-\)/|:\-\*|0:\-\)|;\$|;!|;\-\||;\#|\(\^_\^'\)|:\-\$|:\-@|\(≧∇≦\)|\(T_T\)|\(\*\^0\^\*\)|;\-\*"
abbr_to_slug_cs = {
    r"([A-Z][a-z]+)\.(?: ?(\d+) ?\.( [A-Z]))": r"\1[dot] \2[dot] \3", # Fig. 6. The | Fig. 6.ctct
    r"([A-Z][a-z]+) ?(\d+)\. ?( [A-Z])": r"\1 \2[dot] \3",
    r"([A-Z][a-z]+)\.": r"\1[dot]", # Sentence with one word that starts with captial letter.
}
abbr_to_slug_ci = {
    # r"fig\.(?: ?(\d+)\.)?": r"[fig][dot] \1",
    # r"tab\.(?: ?(\d+)\.)?": r"[tab][dot] \1",
    r"et\.? al\.": "[etal]",
    r"vs\.": "[vs]",
    r"etc\.": "[etc]",
    r"Eq\.": "[Eq]",
}
slug_to_abbr = {
    r"\[dot\]": ".",
    r"\[etc\]": "etc.",
    r"\[vs\]": "vs.",
    r"\[fig\]": "fig",
    r"\[tab\]": "tab",
    r"\[ie\]": "i.e.",
    r"\[sec\]": "sec.",
    r"\[eq\]": "eq.",
    r"\[eg\]": "e.g.",
    r"\[ellipsis\]": "...",
    r"\[aka\]": "a.k.a.",
    r"\[etal\]": "et al."
}
general = [
    r"\( *(?:[\w& \.,*-]+\d{4};?)+ *\)", # citations (Asic et al., 1234)
    r" ?\[\d+( ?, ?\d+)*\]( ?,? ?\[\d+( ?, ?\d+)*\])*", # citations [1,2]; [1]
    r"\(\d+\)( ?, ?\(\d+\))*", # equation numbers (1), (2)
    # r"[αβγδεζηθικλμνξοπρστυφχψωΑΒΓΔΕΖΗΘΙΚΛΜΝΞΟΠΡΣΤΥΦΧΨΩ]",
]
sentence_splitter = py_.flow(
    py_.deburr,
    lambda x: py_.reduce_(
        py_.chain(
            re.findall(r"\b(?:[a-zA-Z]+\.){1,}[a-zA-Z]\.", x)
        ).apply(set)
        .map_(lambda x: (re.sub("\.","\.",x), re.sub("\.","[dot]", x)))
        .from_pairs()
        .value()
        .items(), lambda p, c: re.sub(c[0],c[1], p), x), # a.k.a. i.i.d. e.g. i.e.
    *py_.map_(general, lambda x: sub_ci(x, "")),
    sub_ci(emoticons, ""),
    sub_ci(r",\. ([A-Z0-9])", r". \1"), # cwercwer,. The -> cwercwer. The
    sub_ci(r",\. ?([a-z0-9])", r", \1"), # cwercwer,. cewrc -> cwercwer, cwerc
    sub_ci(r"(\w+)@(\w+)\.(\w+)", r"\1@\2[dot]"),
    sub_ci(r"[\"'] *(.*)([\.\!\?]) *[\"']", r'"\1\"\2'),
    sub_ci(r" *([\.,:])", r"\1"),
    sub_ci(r"\.{3}", "[ellipsis]"),
    sub_ci(r"\.{2}", "."),
    sub_ci(r"\.{4,}", ""),
    sub_ci(r"(?:, ?){2,}", ""),
    sub_ci(r" \)", ")"),
    sub_ci(r"\( ", "("),
    sub_ci(r"([^ \(\.,])\(", r"\1 ("),
    sub_ci(r"\)([^ \)\.,:])", r") \1"),
    sub_ci(r"\/{2,} ", ""),
    sub_ci(r"(\d+)(?:\.(\d+))+", r"\1[dot]\2"),
    *py_.map_(abbr_to_slug_cs.items(), lambda x: sub_cs(x[0], x[1])),
    *py_.map_(abbr_to_slug_ci.items(), lambda x: sub_ci(x[0], x[1])),
    sub_ci(r"(?:\[dot] ){2,}", "[dot]"),
    sub_ci(
        r"arXiv:(\d+)\.(\w+) ?(?:\[(\w+)\.(\w+)\])?", r"arXiv:\1[dot]\2 [\3[dot]\4]"
    ),
    sub_ci(r"\(([^\)]*?)\.([^\)]*?)\)", r"(\1[dot]\2)"),
    sub_ci(r"\[([^\]]*?)\.([^\]]*?)\]", r"[\1[dot]\2]"),
    sub_ci(r"\{([^\}]*?)\.([^\}]*?)\}", r"{\1[dot]\2}"),
    sub_ci(r"\b\d+(\.\d+)*", lambda match: match.group(0).replace(".", "[dot]")),
    py_.strings.clean,
    py_.partial(re.findall, r"[^\.\!\?]*[\.\!\?]"),
    py_.partial(py_.reject, predicate=lambda x: len(x.split(" ")) < 4),
    *py_.map_(
        slug_to_abbr.items(), lambda x: lambda y: py_.map_(y, sub_ci(x[0], x[1]))
    ),
    lambda x: py_.map_(x, lambda y: py_.strings.trim(y)),
)

def group_sentences(sentences, max_tokens=100, overlap=1):
    chunks = []
    tokens_amount = 0
    chunk = []
    for sentence in sentences:
        if tokens_amount < max_tokens:
            chunk.append(sentence)
            tokens_amount += len(py_.strings.words(sentence))
        else:
            chunks.append(chunk)
            chunk = chunk[len(chunk) - overlap :] + [sentence]
            tokens_amount = py_.reduce_(chunk, lambda total, sentence: len(py_.strings.words(sentence)) + total, 0)
    else:
        chunks.append(chunk)

    return py_.chain(chunks[bool(overlap) :]).map_(lambda x: " ".join(x)).value()

pretty_print_with_zeros = lambda number, length: str(number).zfill(length)

In [ ]:
prepare_rerank_model = lambda x: Ranker(
    model_name={
        0: "ms-marco-TinyBERT-L-2-v2",
        1: "ms-marco-MiniLM-L-12-v2",
        2: "rank-T5-flan",
    }[x],
    cache_dir="/opt",
)

rerank_passages = py_.flow(
    lambda model, query, passages: {
        "model": model,
        "rerankrequest": RerankRequest(query=query, passages=passages),
    },
    lambda kwargs: kwargs["model"].rerank(kwargs["rerankrequest"]),
    lambda x: py_.map_(x, lambda y: y["text"]),
)

In [ ]:
%%capture
embedder = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
service_account_file_name = "./drive/MyDrive/Thesis/service_account_key.json"

credentials = service_account.Credentials.from_service_account_file(
    service_account_file_name
)

scoped_credentials = credentials.with_scopes(
    [
        "https://www.googleapis.com/auth/cloud-platform",
        "https://www.googleapis.com/auth/generative-language.retriever",
    ]
)

generative_service_client = glm.GenerativeServiceClient(credentials=scoped_credentials)

# Actual code

In [ ]:
prepare_grounding_passages= lambda docs: glm.GroundingPassages(passages=py_.chain(docs).map_(lambda doc: glm.Content(parts=[glm.Part(text=doc)])).map_(lambda passage, index: glm.GroundingPassage(content=passage, id=f"{index}")).value())

In [ ]:
def prepare_corpus(chunks, keywords, regex=True):
    isKeywordInChunk = lambda chunk, keyword: re.search(keyword if regex else re.escape(keyword), chunk, re.IGNORECASE)
    isChunkUseful = lambda chunk: py_.some(keywords, py_.partial(isKeywordInChunk,chunk))
    return py_.chain(chunks).filter_(isChunkUseful).value()

def resolve_hit_documents(corpus, query_hits):
    indices_filtered_corpus = py_.chain(query_hits).flatten().map_(lambda hit: hit["corpus_id"]).map_(int).value()
    return py_.chain(corpus).filter_(lambda _, index: index in indices_filtered_corpus).value()


def rerank_(model_id=0):
    rerank_model = prepare_rerank_model(model_id)

    def f(docs):
        rerank_query = "dataset usage in retrieved documents."
        docs = rerank_passages(rerank_model, rerank_query, [{"text": d} for d in docs])
        return docs

    return f

rerank_documents = rerank_()

prepare_embeddings = lambda texts: embedder.encode(texts, convert_to_tensor=True)

ms-marco-TinyBERT-L-2-v2.zip: 100%|██████████| 3.26M/3.26M [00:00<00:00, 57.9MiB/s]


In [ ]:
prepare_query_content = py_.flow(
    lambda add_to_query = "": """Extract all named datasets used or mentioned in the provided passages from a research paper as it is.
Do not change or modify the extracted dataset.
Please ensure that the output is in csv format and that only datasets with explicit names are included from the passages.
For clarity, a dataset refers to a collection of organized data points or records that serve a specific purpose.
Datasets are commonly utilized in various fields such as science, research, machine learning, statistics, economics, and more.
They can be structured or unstructured and are often referenced in research papers to support findings, validate hypotheses, or provide evidence for arguments.
Datasets may be explicitly mentioned within the passages, such as "We utilize the <Dataset> collected from <Source> for our analysis." or "The <Dataset> provided by <Provider> contains valuable information for our research."
Additionally, datasets can be constructed from other datasets through aggregation, transformation, or combination processes.
For instance, "We constructed our dataset by merging data from multiple sources, including <Dataset1> and <Dataset2>."
In some cases, the word "dataset" may be implicit, and datasets may be referred to by other terms such as "data collection", "data source", or "data repository".
Ensure that the extraction process accounts for variations in terminology and identifies datasets based on context and proximity to related terms.
Do not consider datasets having "=" in the name.
Ensure that the extraction process focuses on identifying datasets with specific names and excludes general descriptions of data sources or collections. Datasets are alphanumeric words that may not have any meaning.
""" + add_to_query,
    lambda user_query: glm.Part(text=user_query),
    lambda part: glm.Content(parts=[part])
)

"""
Extract named datasets from the provided passages in csv format.
Datasets are collections of organized data points used across various fields.
They can be mentioned explicitly or constructed from other datasets.
Account for variations in terminology and context, but surely extract the datasets which are followed by the word "dataset".
Datasets are alphanumeric words without inherent meaning.
In cases where names may be confusingly similar, such as a dataset and a model sharing the same name, ensure that both datasets and models with similar names are extracted.
"""

generate_answer = py_.flow(
    lambda grounded_passages, query_content, temperature=None: glm.GenerateAnswerRequest(
        model="models/aqa",
        contents=[query_content],
        inline_passages=grounded_passages,
        temperature=temperature,
        answer_style="EXTRACTIVE",  # or ABSTRACTIVE, EXTRACTIVE, VERBOSE
    ),
    generative_service_client.generate_answer
)

In [ ]:
regex_keywords_phrases = ['data(set|base)',
    'anal(ytics|ysis)',
    'resear(ch|ch paper)',
    'stud(y|ies?)',
    'exper(iment|iments?)',
    'method(ology|ologies?)',
    'collect(ion|ions?)',
    'sampl(e|ing)',
    'variabl(e|es?)',
    'observ(ation|ations?)',
    'surve(y|ys?)',
    'popul(ation|ations?)',
    'repositor(y|ies?)',
    'databas(e|es?)',
    'sourc(e|es?)',
    'raw data',
    'secondar(y|ies?)',
    'primar(y|ies?)',
    'min(e|ing)',
    'proces(s|sing)',
    'clean(ing|)',
    'manipul(ation|ations?)',
    'integrat(e|ion)',
    'aggregat(e|ion)',
    'visualiz(e|ation)',
    'interpret(ation|ations?)',
    '(used|employed|utilized) for (analysis|modeling|evaluation|research)',
    '(trained|experimented) on',
    'analy(zed|sis) (data|dataset)',
    '(examined|derived|investigated|explored) (data|dataset)',
    '(employed|modeled) with (data|dataset)',
    '(evaluated|tested|compared) on',
    '(referenced|applied) (dataset|data)',
    '(accessed|reviewed) (data|dataset) from',
    "data(-|\s)?set",
    "task",
    "challenge",
    "(knowledge|data)\s*base",
    "benchmark",
    "(experiment|train|performance)[\sa-zA-Z0-9]+on",
    "corpus",
    "class",
    "(train|test)[\sa-zA-Z0-9]+(set)?",
]

queries = [
    "Data used in the study",
    "Datasets employed for analysis",
    "Data sources referenced",
    "Dataset utilized for research",
    "Data collection methods",
    "Datasets examined in the paper",
    "Data analysis conducted",
    "Datasets referenced in the research",
    "Data sources investigated",
    "Dataset mentioned in the study",
    "Data utilized for analysis",
    "Datasets considered in the research",
    "Data collection procedures",
    "Dataset discussed in the paper",
    "Data sources utilized",
    "Datasets referenced for analysis",
    "Data used for research purposes",
    "Dataset examined in the study",
    "Data sources referenced in the paper",
    "Datasets employed for investigation"
]

# queries = [
    # "datasets, database, knowledgebase",
    # "experimented, trained, tested, performance on.",
    # "corups.",
    # "class.",
    # "task, challenge",
# ]

# queries = [
#     "Supporting Findings",
#     "Validation of Hypotheses",
#     "Empirical Analysis",
#     "Comparative Studies",
#     "Replication and Verification",
#     "Methodological Transparency",
#     "Citation and Attribution",
#     "Benchmarking",
#     "Standardization and Comparison",
#     "Community Collaboration"
# ]


query_embeds = prepare_embeddings(queries)

In [ ]:
id_range = [200,300]
temp_range = [291,292]
current_id = id_range[0] - 1

# GLOBAL_DATASETS = {}
datasets = set()

for_range = temp_range

for current_id in tqdm.tqdm(range(*for_range)):
    chunks = py_.flow(
        lambda x: get_text(dmdd, x),
        sentence_splitter,
        lambda x: group_sentences(x, 200, 2),
    )(current_id)

    dataset_keywords = []

    while True:
        keywords = regex_keywords_phrases if len(datasets) < 1 else dataset_keywords

        corpus = prepare_corpus(chunks, keywords=keywords, regex=len(datasets) < 1)
        corpus_embeds = prepare_embeddings(corpus)

        queries_hits = util.semantic_search(query_embeds, corpus_embeds, top_k=20)
        docs = resolve_hit_documents(corpus, queries_hits)

        grounding_passages = prepare_grounding_passages(docs)

        query_content = prepare_query_content(
            "Example datasets found are: {}.".format(", ".join(datasets)) if len(datasets) > 0 else ""
        )


        response = generate_answer(grounding_passages, query_content)
        attempted_answer = py_.attempt(lambda _: response.answer.content.parts[0].text, None)
        ic(attempted_answer)

        if (py_.is_error(attempted_answer)):
            print(attempted_answer)
            break

        for text_in_brackets in re.findall(r"\((.*?)\)", attempted_answer):
            if not re.search(r"\( *(?:[\w& \.,*-]+\d{4};?)+ *\)", text_in_brackets):
                continue
            attempted_answer = re.sub(rf"\({text_in_brackets}\)", "", attempted_answer)

        temp_baselines = py_.chain(attempted_answer.split(", ")).map_(lambda x: x.strip()).filter_(lambda x: len(x.split(" ")) < 7).apply(list).value()
        temp_baselines = datasets.union(temp_baselines)

        if temp_baselines - datasets == set():
            break

        datasets = temp_baselines
        dataset_keywords = datasets

        for dataset in datasets:
            if m := re.findall(r"\((.*?)\)", dataset):
                m = [_.strip() for _ in m]
                dataset_keywords = dataset_keywords.union(m)
                dataset_keywords = dataset_keywords.union(
                    {re.sub(rf"\({_}\)", "", dataset).strip() for _ in m}
                )

    GLOBAL_DATASETS[current_id] = py_.chain(datasets).map_(sub_ci(" +"," ")).filter_(lambda x: len(x) > 0).apply(list).value()
    datasets = set()

  0%|          | 0/1 [00:00<?, ?it/s]ic| attempted_answer: 'Continuous Hopfield Network'
ic| attempted_answer: 'input data set, x (t)'
ic| attempted_answer: 'x (t), Continuous Hopfield Network, input data set'
100%|██████████| 1/1 [00:49<00:00, 49.34s/it]


In [ ]:
GLOBAL_DATASETS

{200: ['MatchNMingle', 'SALSA'],
 201: ['SUNCG', 'MVS', 'SUN3D'],
 202: ['MARS', 'PRID 2011', 'iLIDS-VID'],
 203: ['The People-Snapshot dataset', 'the dataset used in'],
 204: ['train', 'test1&2', 'development'],
 205: ['IBUG', 'COFW', 'AFLW'],
 206: ['Multi-View Stereo Correspondence (MVS-Corr)', 'HPatches'],
 207: ['ShapeNet', 'SHAPENET'],
 208: ['HAM10000 Dataset'],
 209: ['CIFAR-10', 'ImageNet-100'],
 210: ['PASCAL VOC', 'MS-COCO'],
 211: ['FlyingChairs', 'FlyingThings3D', 'ChairsSDHom'],
 212: ['Amazon Reviews dataset',
  'IMDB movie review datasets',
  'Binary Stanford Sentiment Treebank (SST)'],
 213: ['T matrix',
  'SqueezeDet',
  'T overlap',
  'X overlap',
  'Driving in the Matrix',
  'X matrix'],
 214: ['sketch perceptual grouping (SPG) dataset'],
 215: ['ORL', 'YALE', 'MNIST'],
 216: ['Im2GPS3k', 'Im2GPS test set', 'Im2GPS', 'YFCC4k'],
 217: ['Caltech-USA dataset', 'Caltech10 training set', 'KITTI dataset'],
 218: ['x-shaped Gaussian', 'banana distribution', 'multimodal Gau

In [ ]:
# id_range = [0,100]

with open(
    f"./drive/MyDrive/Thesis/Dataset_Extraction/raw dataset/dmdd_modified/{id_range[0]}_{id_range[1] - 1}.pkl",
    "wb",
) as file:
    pickle.dump(file=file, obj=GLOBAL_DATASETS, protocol=pickle.HIGHEST_PROTOCOL)


# with open(
#     f"./drive/MyDrive/Thesis/Dataset_Extraction/raw dataset/dmdd_modified/{id_range[0]}_{id_range[1] - 1}.pkl",
#     "rb",
# ) as file:
#     GLOBAL_DATASETS = pickle.load(file=file)

In [ ]:
# @title function to generate scores
def score_fn(observed, ground) -> dict[str,None|set]:

    stop_words = ["the"]

    def f(text):
        text = sub_ci("(data ?set|database|corpus|caption|benchmark|train|test).*","")(text)
        text = py_.flow(*py_.chain(stop_words).map_(lambda x: sub_ci(x, "")).value())(text)
        return re.sub(r" +"," ",text).strip()

    original = {
        "ground": {f(text).lower(): text for text in ground},
    }

    temp = {}
    for o in observed:
        m = re.findall(r"\((.*)\)",o.lower())
        if len(m) > 0:
            temp[f(m[0])] = o
            d_ = f(o.lower().replace(f"({m[0]})","").strip())
            if len(d_) > 0:
                temp[d_] = o
                if len(d_.split(" ")) > 1:
                    temp[py_.chain(d_.split(" ")).map_(lambda x: x[0].upper()).apply(py_.strings.join).value()] = o
        else:
            if len(d_ := f(o.lower())) > 1:
                temp[d_] = o
                if len(d_.split(" ")) > 1:
                    temp[py_.chain(d_.split(" ")).map_(lambda x: x[0]).apply(py_.strings.join).value()] = o

    original['observed'] = temp

    ground = set(map(f, original['ground'].keys()))
    observed = set(original['observed'].keys())


    # TP: True Positives (correctly predicted positive instances)
    # TN: True Negatives (correctly predicted negative instances)
    # FP: False Positives (incorrectly predicted as positive instances)
    # FN: False Negatives (incorrectly predicted as negative instances)


    fp = set()
    tp = set()
    temp = set()
    for g in ground:
        for o in observed:
            if  o == g :
                # ic(o,g)
                tp.add(original['ground'][g])
                temp.add(g)
                break

    for o in observed:
        for g in ground:
            if o == g:
                # ic(o,g)
                break
        else:
            # ic(o)
            if original['observed'][o] not in tp:
                fp.add(original['observed'][o])

    # ic(tp, fp)

    new = fp
    observed = observed.union(temp)

    tp = len(tp)
    # tp = len(ground.intersection(observed))
    fp = len(fp)
    # fp = len(observed - ground)
    tn = 0
    fn = len(ground - observed)


    if tp+fp == 0:
        precision = None
    else:
        precision = tp / (tp + fp)

    if tp+fn == 0:
        recall = None
    else:
        recall = tp / (tp + fn)

    if tp+tn+fp+fn == 0:
        accuracy = None
    else:
        accuracy = (tp+tn) / (tp + tn + fp + fn)

    ground_ds = list(map(lambda x: re.sub(" +", " ", x), original['ground'].values()))
    missed_ds = list(map(lambda x: re.sub(" +", " ", x), {original['ground'][text] for text in ground - observed}))
    new_ds = list(map(lambda x: re.sub(" +", " ", x), new))

    metric_scores = {
        "precision": precision,
        "recall": recall,
        "accuracy": accuracy,

        "ground": ground_ds if len(ground_ds) > 0 else None,
        "missed": missed_ds if len(missed_ds) > 0 else None,
        "new": new_ds if len(new_ds) > 0 else None,
    }
    return metric_scores

In [ ]:
statistics = {}
for id, datasets in tqdm.tqdm(GLOBAL_DATASETS.items()):
    # if id == 190:
    statistics[id] = score_fn(datasets, get_true_datasets(dmdd, id))

df = pd.DataFrame(statistics).T
# df.index.names = ["DMDD ID"]

100%|██████████| 99/99 [00:00<00:00, 1768.35it/s]


In [ ]:
df

,precision,recall,accuracy,ground,missed,new
200,0.5,1.0,0.5,[SALSA],None,[MatchNMingle]
201,0.0,0.0,0.0,"[KITTI, TUM RGB-D]","[TUM RGB-D, KITTI]","[SUNCG, MVS, SUN3D]"
202,0.0,0.0,0.0,[ImageNet],[ImageNet],"[MARS, PRID 2011, iLIDS-VID]"
203,1.0,1.0,1.0,[People-snapshot dataset],None,None
204,0.0,None,0.0,None,None,[development]
...,...,...,...,...,...,...
295,0.5,1.0,0.5,[FEVER],None,[Emergent]
296,0.0,None,0.0,None,None,"[Resource Management (RM), Wall Street Journal..."
297,0.5,1.0,0.5,[Fashion MNIST],None,[MNIST]
298,0.333333,0.5,0.25,"[ViZDoom, ViZ-Doom]",[ViZ-Doom],"[VOT, UE]"


In [ ]:
df.to_excel(f"./drive/MyDrive/Thesis/Dataset_Extraction/statistics/dmdd_modified/{id_range[0]}_{id_range[1] - 1}.xlsx")

In [ ]:
datasets_detected = []

# for dmdd_id in [54]:
for dmdd_id in tqdm.tqdm(df.T):
    sentences = sentence_splitter(get_text(dmdd, dmdd_id))
    chunks = group_sentences(sentences, 20, 0)
    if df.T[dmdd_id]["new"] == None:
        continue
    for dataset in df.T[dmdd_id]["new"]:
        context_arr = set()
        for i, chunk in enumerate(chunks):
            context = []
            k = 0
            while True:
                if dataset not in chunk:
                    break
                context = chunks[
                    0 if i < (2 - k) else i - (2 - k) : (
                        len(chunks)
                        if i > len(chunks) - (3 - k)
                        else i + (3 - k)
                    )
                ]
                if len(" ".join(context)) <= 700 or k > 1:
                    break
                k += 1

            if len(context) > 0: context_arr.add(" ".join(context))
        datasets_detected.append({"dmdd_id": dmdd_id, "new": dataset, "context": list(context_arr)})

99it [00:08, 11.23it/s]


In [ ]:
exploded_df: pd.DataFrame = pd.DataFrame(datasets_detected).explode("context")
exploded_df["original"] = exploded_df["dmdd_id"].map(lambda x: ", ".join(y if (y:=df["ground"][x]) != None else []) if len(", ".join(y if (y:=df["ground"][x]) != None else [])) > 0 else None)
exploded_df["DR"] = ""
exploded_df["PD"] = ""
exploded_df["Source"] = ""
exploded_df = exploded_df.set_index(
    [
        "dmdd_id",
        "original",
        "new",
        "DR",
        "PD",
        "Source",
        exploded_df.groupby(["new"]).cumcount(),
    ]
)
exploded_df.index.names = ["DMDD ID", "Original", "New", "DR", "PD", "Source", "Passage No."]
exploded_df.to_excel(
    f"./drive/MyDrive/Thesis/Dataset_Extraction/annotations/dmdd_modified/{id_range[0]}_{id_range[1] - 1}.xlsx"
)
# exploded_df.isna()
# exploded_df.isna().sum()

In [ ]:
# @title Format code
!black ./drive/MyDrive/'Colab Notebooks'/'Baseline Presentation.ipynb'

In [ ]:
# for text in group_sentences(sentence_splitter(get_text(dmdd,1)), 20, 0):
#     print("Burst".lower() in text.lower())
sentence_splitter(get_text(dmdd, 1))
# for i, text in enumerate(group_sentences(sentences, 20, 0)):
#     if "Burst" in text:
#         print(i)